In [2]:
import numpy as np
import matplotlib.pyplot as plt
X_train = (np.load("../data/data_selected.npy"))[:,:]
labels = []
for i in range(1,20):
    for j in range(0,360):
        labels.append(i)
y_train = np.array(labels)
X_test = (np.load("../data/data_test_selected.npy"))[:,:]
print(X_train.shape)
print(X_test.shape)
labels = np.asarray(labels)
print(labels.shape)

(855000, 13)
(285000, 13)
(6840,)


In [3]:
def groupedAvg(myArray, N=2):
    result = np.cumsum(myArray, 0)[N-1::N]/float(N)
    result[1:] = result[1:] - result[:-1]
    return result
def groupedVar(original_array,N):
    newList = []
    for i in range(0, original_array.shape[0], N):
        r = original_array[i:i+N,:].var(axis=0)
        newList.append(r)
    new_array = np.asarray(newList)
    return new_array

X_train1 = groupedAvg(X_train,N=125)
X_train2 = groupedVar(X_train,N=125)
X_train = np.hstack((X_train1,X_train2))
X_train

array([[-7.90726080e-01, -6.84903440e-02,  1.35896800e-01, ...,
         7.09758042e-07,  4.96449755e-05,  6.21709094e-07],
       [-7.91961680e-01, -7.01570000e-02,  1.37123040e-01, ...,
         8.95215834e-07,  5.06765722e-05,  7.34423040e-07],
       [-7.92875360e-01, -7.07231760e-02,  1.36436400e-01, ...,
         8.05900442e-07,  4.25671452e-05,  5.75610240e-07],
       ...,
       [-8.05772640e-01,  3.80417680e-01, -4.46727200e-01, ...,
         4.93688000e-02,  6.21623567e-01,  1.70895329e-02],
       [-7.57702000e-01,  4.12746480e-01, -5.56242720e-01, ...,
         4.01646318e-02,  3.88494671e-01,  1.80311894e-02],
       [-5.65478904e-01,  1.56690368e-01, -7.96862800e-01, ...,
         1.41987731e-02,  4.54050084e-01,  9.29973933e-03]])

In [4]:
from sklearn import svm
from sklearn.svm import SVC

clf = svm.SVC(kernel='rbf',class_weight='balanced',decision_function_shape='ovo')
clf.cache_size = 500
# clf = svm.SVC(kernel='rbf', C=3, gamma='scale')
# clf.cache_size = 500
# clf.fit(X_train, labels)
#
# print("training score:", clf.score(X_train, labels))

In [5]:
X_test1 = groupedAvg(X_test, N=125)
print(X_test1.shape)
X_test2 = groupedVar(X_test, N=125)
X_test = np.hstack((X_test1, X_test2))
print(X_test.shape)

(2280, 13)
(2280, 26)


In [6]:
labels_test = []
for i in range(1,20):
    for j in range(0,120):
        labels_test.append(i)
labels_test = np.asarray(labels_test)
labels_test

array([ 1,  1,  1, ..., 19, 19, 19])

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
c_range = np.logspace(-5, 15, 21, base=2)
gamma_range = np.logspace(-9, 3, 13, base=2)
param_grid = [{'kernel': ['rbf'], 'C': c_range, 'gamma': gamma_range}]
grid = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1,refit=True)
clf = grid.fit(X_train, labels)
score_test = grid.score(X_test, labels_test)
score_train = grid.score(X_train, labels)
print('test accuracy%s' % score_test)
print('train accuracy%s' % score_train)

test accuracy0.8232456140350877
train accuracy0.9931286549707602


In [8]:
grid.best_params_

{'C': 64.0, 'gamma': 0.0078125, 'kernel': 'rbf'}

In [34]:
clf = svm.SVC(kernel='linear', C=64, gamma=grid.best_params_['gamma'],decision_function_shape='ovo')
clf.cache_size = 500
clf.fit(X_train, labels)

print("X_train score:", clf.score(X_train, labels))
labels_predict_svm = clf.predict(X_test)
print(labels_predict_svm)
print(labels_test)
errorssvm = np.nonzero(labels_predict_svm - labels_test)
print("X_test error：", len(errorssvm[0]))
print("error account for：",len(errorssvm[0])/len(labels_test))

X_train score: 0.9940058479532163
[ 1  1  1 ... 19 19 19]
[ 1  1  1 ... 19 19 19]
X_test error： 427
error account for： 0.18728070175438596


In [35]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=10000,multi_class='multinomial',solver='sag')
lr_model.fit(X_train,labels)
print("training score:", lr_model.score(X_train, labels))
labels_predict = lr_model.predict(X_test)
print(labels_predict)
print(labels_test)
errors = np.nonzero(labels_predict - labels_test)
print("X_test error：", len(errors[0]))
print("error account for：",len(errors[0])/len(labels_test))

training score: 0.9880116959064328
[ 1  1  1 ... 19 19 19]
[ 1  1  1 ... 19 19 19]
X_test error： 408
error account for： 0.17894736842105263


In [36]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train, labels)
rfc.score(X_test, labels_test)
print("training score:", lr_model.score(X_train, labels))
labels_predict_forest = lr_model.predict(X_test)
print(labels_predict_forest)
print(labels_test)
errors = np.nonzero(labels_predict_forest - labels_test)
print("X_test error：", len(errors[0]))
print("error account for：",len(errors[0])/len(labels_test))

training score: 0.9880116959064328
[ 1  1  1 ... 19 19 19]
[ 1  1  1 ... 19 19 19]
X_test error： 408
error account for： 0.17894736842105263


In [30]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print(accuracy_score(labels_test,labels_predict_svm))
print(precision_score(labels_test,labels_predict_svm,average='macro'))
print(recall_score(labels_test,labels_predict_svm,average='macro'))

0.8127192982456141
0.8328243396936014
0.8127192982456141


D:\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
err_list = []
for i in range(0,2280):
    if labels_predict_svm[i]!=labels_test[i]:
        err_list.append((labels_predict_svm[i],labels_test[i]))
err_list = np.asarray(err_list)
unique1,count1=np.unique(err_list.T[0],return_counts=True)
unique2,count2=np.unique(err_list.T[1],return_counts=True)
data_count1=dict(zip(unique1,count1))
data_count2=dict(zip(unique2,count2))
print(data_count1)
print(data_count2)

{2: 10, 5: 14, 6: 6, 7: 165, 8: 126, 9: 3, 10: 30, 12: 1, 13: 33, 14: 3, 19: 12}
{1: 60, 2: 56, 4: 60, 6: 25, 7: 50, 8: 45, 9: 34, 10: 2, 11: 30, 13: 4, 14: 1, 16: 20, 18: 7, 19: 9}


In [32]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print(accuracy_score(labels_test,labels_predict_forest))
print(precision_score(labels_test,labels_predict_forest,average='macro'))
print(recall_score(labels_test,labels_predict_forest,average='macro'))

0.8210526315789474
0.8586141537668027
0.8210526315789474
